In [1]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
%matplotlib inline

from linalg_helper import make_random_direction, make_random_diagonal_matrix

In [2]:
def identify(A, x0, make_controller):
    controller = make_controller(x0)
    N = len(x0)
    x = x0
    for i in range(N + 1):
        direction = make_random_direction(N)
        controller.observe(direction, x.T @ direction, i)
        print(f"Estimated eigenvalues: {controller.estimate()}")
        print(f"True eigenvalues: {np.linalg.eigvals(A)}")
#         print(f"x: {x}")
        print()
        x = A @ x

In [3]:
class LeastSquaresController:
    def __init__(self, x0):
        self.N = len(x0)
        self.x0 = x0
        self.observations = []
        self.best = []
        
    class Observation:
        def __init__(self, x0, coeffs, val, order):
            self.x0 = x0
            self.coeffs = coeffs
            self.val = val
            self.order = order
            print(coeffs, val, order)
            
        def __repr__(self):
            return f"Observation({self.coeffs}, {self.val}, {self.order})"
            
        def error(self, guess):
            curr = 0
            for xi, lamb, coeff in zip(self.x0, guess, self.coeffs):
                curr += (lamb ** self.order) * xi * coeff
            return self.val - curr
    
    def observe(self, direction, proj, i):
        if i == 0:
            return
        self.observations.append(self.Observation(self.x0, direction, proj, i))
    
    def estimate(self):
        if not self.observations:
            return
        def cost(guess):
            return [observation.error(guess) for observation in self.observations]
        out = opt.least_squares(cost, np.hstack([self.best, [0]]))
        self.best = out.x
        return self.best

In [4]:
N = 10
identify(make_random_diagonal_matrix(N, 5, 0.2), 
         np.ones(N),
         LeastSquaresController)

Estimated eigenvalues: None
True eigenvalues: [5.09371869e+00 9.36020090e-01 1.93591829e-01 4.24444591e-02
 8.13674212e-03 1.55686068e-03 3.47596658e-04 6.47946155e-05
 1.26994892e-05 2.59938932e-06]

[ 0.2365422  -0.39981212  0.13510772 -0.0727511  -0.19616304 -0.07555869
  0.59957308 -0.25327182 -0.13329061 -0.52446287] 0.8521902725880045 1
Estimated eigenvalues: [3.60269875]
True eigenvalues: [5.09371869e+00 9.36020090e-01 1.93591829e-01 4.24444591e-02
 8.13674212e-03 1.55686068e-03 3.47596658e-04 6.47946155e-05
 1.26994892e-05 2.59938932e-06]

[-0.03878535  0.44660051  0.24354445  0.15438918 -0.48833992 -0.11224604
  0.1816622   0.54648068 -0.34414239  0.12141843] -0.6056688586160719 2
Estimated eigenvalues: [4.05442515 0.26725641]
True eigenvalues: [5.09371869e+00 9.36020090e-01 1.93591829e-01 4.24444591e-02
 8.13674212e-03 1.55686068e-03 3.47596658e-04 6.47946155e-05
 1.26994892e-05 2.59938932e-06]

[ 0.27232645  0.20916002 -0.48390214 -0.37400853  0.21982393 -0.06033929
  0.3964